In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import *
from sklearn.linear_model import *
from math import *
from nltk import *
import warnings
warnings.filterwarnings("ignore")
from deep_translator import *
import tensorflow as tf
import os
import pathlib as pl
from nltk.sentiment.vader import *
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from textblob import TextBlob
import nltk
from transformers import pipeline

In [2]:
download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/amith/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to
[nltk_data]    |

True

In [3]:
df_train = pd.read_csv("train_translated.csv")
df_test = pd.read_csv("test_translated.csv")

In [4]:
df_train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Little things like that make a huge difference...,I was trying to accomplish something.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,In role play as well. Opportunities to express...,Children can see how different ethnicities are.,th,Thai,1


In [5]:
df_test.head()

,id,premise,hypothesis,lang_abv,language
0,c6d58c3f69,The boxes will be marked with the names of Buc...,"There will be no memorial for Casey, one of th...",ur,Urdu
1,cefcc82292,This is what we have been advised.,"When told what to do, management failed to let...",ar,Arabic
2,e98005252c,and this is largely due to mothers taking drugs,Mothers take drugs.,fr,French
3,58518c10ba,Dialogue with representatives of the city and ...,The IMA works with other organizations because...,zh,Chinese
4,c32b0d16df,She was still there.,"We thought she was gone, however, she stayed.",ru,Russian


In [6]:
df_train.isna().sum()

id            0
premise       0
hypothesis    0
lang_abv      0
language      0
label         0
dtype: int64

In [7]:
df_test.isna().sum()

id            0
premise       0
hypothesis    0
lang_abv      0
language      0
dtype: int64

In [8]:
df_train_new = df_train.drop(["id","lang_abv","language"],axis=1,inplace=False)
df_test_new = df_test.drop(["id","lang_abv","language"],axis=1,inplace=False)

In [9]:
df_train_new

,premise,hypothesis,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Little things like that make a huge difference...,I was trying to accomplish something.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,In role play as well. Opportunities to express...,Children can see how different ethnicities are.,1
...,...,...,...
12115,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,2
12116,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,0
12117,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",2
12118,At the west end is a detailed model of the who...,The model temple complex is at the east end.,2


In [18]:
sia = SentimentIntensityAnalyzer()
y = lambda x: sia.polarity_scores(text=x)

In [29]:
df_train_polar = df_train_new["label"]
df_test_polar = pd.DataFrame()

In [31]:
h1 = map(y,df_train_new["premise"])
list(h1)

[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.709, 'pos': 0.291, 'compound': 0.5456},
 {'neg': 0.0, 'neu': 0.751, 'pos': 0.249, 'compound': 0.6077},
 {'neg': 0.0, 'neu': 0.653, 'pos': 0.347, 'compound': 0.8807},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.076, 'neu': 0.756, 'pos': 0.169, 'compound': 0.4939},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.191, 'neu': 0.809, 'pos': 0.0, 'compound': -0.5499},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.231, 'neu': 0.385, 'pos': 0.385, 'compound': 0.296},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.794, 'pos': 0.206, 'compound': 0.6369},
 {'neg': 0.159, 'neu': 0.841, 'pos': 0.0, 'compound': -0.631},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.388, 'neu': 0.518, 'pos': 0.094, 

In [23]:
df_train_polar.head()

,neg,neu,pos,compound,label
0,0.0,1.000,0.000,0.0000,0
1,0.0,1.000,0.000,0.0000,2
2,0.0,0.588,0.412,0.4215,0
3,0.0,1.000,0.000,0.0000,0
4,0.0,1.000,0.000,0.0000,1


In [71]:
df_test_polar.head()

,neg,neu,pos
0,0.078,0.922,0.0
1,0.000,1.000,0.0
2,0.000,1.000,0.0
3,0.000,1.000,0.0
4,0.000,1.000,0.0
